In [1]:
import gc
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
!ls /Users/wizardholy/Documents/corpus/porto-seguro-safe-driver-prediction

sample_submission.csv test.csv              train.csv


In [3]:
dir = '/Users/wizardholy/Documents/corpus/porto-seguro-safe-driver-prediction'
train_path = dir+'/train.csv'
test_path = dir+'/test.csv'

In [4]:
trainDf = pd.read_csv(train_path)

In [5]:
trainDf.head(4)

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0


In [6]:
pos_trainDf = trainDf[trainDf['target'] == 1]
neg_trainDf = trainDf[trainDf['target'] == 0].sample(n=20000, random_state=2000)

In [7]:
ftrainDf = pd.concat([pos_trainDf, neg_trainDf], axis=0).sample(frac=1.0, random_state=2018)

In [8]:
del trainDf
del pos_trainDf
del neg_trainDf
gc.collect()

0

In [9]:
print(ftrainDf.shape, ftrainDf['target'].mean())

(41694, 59) 0.5203146735741354


In [10]:
X_train, X_test, y_train, y_test = train_test_split(ftrainDf, ftrainDf['target'], test_size=0.25, random_state=2017)

In [11]:
bin_feats = ['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin']
cat_feats = ['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat']
con_feats = ['ps_ind_01', 'ps_ind_03', 'ps_ind_14', 'ps_ind_15', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14']

In [12]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [13]:
train_sz = X_train.shape[0]

In [14]:
combineDf = pd.concat([X_train, X_test], axis=0)

In [15]:
del X_train
del X_test
gc.collect()

0

In [16]:
from sklearn.preprocessing import MinMaxScaler

In [17]:
#连续特征全部归一化
for col in con_feats:
    scaler = MinMaxScaler()
    combineDf[col] = scaler.fit_transform(np.array(combineDf[col].values.tolist()).reshape(-1,1))

In [18]:
# 离散特征one-hot
for col in cat_feats:
    onehotret = pd.get_dummies(combineDf[col], prefix=col)
    combineDf = pd.concat([combineDf, onehotret], axis=1)

In [19]:
combineDf.tail(10)

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_car_11_cat_95,ps_car_11_cat_96,ps_car_11_cat_97,ps_car_11_cat_98,ps_car_11_cat_99,ps_car_11_cat_100,ps_car_11_cat_101,ps_car_11_cat_102,ps_car_11_cat_103,ps_car_11_cat_104
162165,405359,1,0.428571,1,0.545455,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
590862,1476888,1,0.000000,3,0.363636,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
563584,1409097,0,0.428571,1,0.636364,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
71372,178520,1,1.000000,1,0.727273,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
233795,584100,1,0.000000,1,0.454545,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
323872,809455,0,0.000000,1,0.363636,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
134468,335949,0,0.142857,1,1.000000,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
368853,921700,0,0.285714,1,0.090909,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
542034,1354617,1,0.142857,1,0.363636,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
15592,39159,0,0.142857,3,1.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# 训练模型
label = 'target'
onehotret_feats = [col for col in combineDf.columns if col not in ['id', 'target'] + con_feats+cat_feats+bin_feats]

In [21]:
train_sz

31270

In [22]:
train = combineDf[:train_sz]
test = combineDf[train_sz:]

In [23]:
print("Train.shape: {0}, Test.shape: {1}".format(train.shape, test.shape))
del combineDf

Train.shape: (31270, 242), Test.shape: (10424, 242)


In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [25]:
# LR模型
lr_feats = con_feats + onehotret_feats
lr = LogisticRegression()
lr.fit(train[lr_feats], train[label].values)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
help(classification_report)

Help on function classification_report in module sklearn.metrics._classification:

classification_report(y_true, y_pred, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False, zero_division='warn')
    Build a text report showing the main classification metrics
    
    Read more in the :ref:`User Guide <classification_report>`.
    
    Parameters
    ----------
    y_true : 1d array-like, or label indicator array / sparse matrix
        Ground truth (correct) target values.
    
    y_pred : 1d array-like, or label indicator array / sparse matrix
        Estimated targets as returned by a classifier.
    
    labels : array, shape = [n_labels]
        Optional list of label indices to include in the report.
    
    target_names : list of strings
        Optional display names matching the labels (same order).
    
    sample_weight : array-like of shape (n_samples,), default=None
        Sample weights.
    
    digits : int
        Number of digits for for

In [1]:
def do_model_metric(y_true, y_pred, y_pred_prob):
    print(classification_report(y_true, y_pred))
    print("Predict 1 percent: {0}".format(np.mean(y_pred)))
    print("Label 1 percent: {0}".format(train[label].mean()))
    from sklearn.metrics import roc_auc_score,accuracy_score
    print("AUC: {0:.3}".format(roc_auc_score(y_true=y_true, y_score=y_pred_prob[:,1])))
    print("Accuracy: {0}".format(accuracy_score(y_true=y_true, y_pred=y_pred)))
print("Train............")
do_model_metric(y_true=train[label], y_pred=lr.predict(train[lr_feats]), y_pred_prob=lr.predict_proba(train[lr_feats]))


Train............


NameError: name 'train' is not defined

In [33]:
print("\n\n")
print("Test.............")
do_model_metric(y_true=test[label], y_pred=lr.predict(test[lr_feats]), y_pred_prob=lr.predict_proba(test[lr_feats]))




Test.............
             precision    recall  f1-score   support

          0       0.58      0.53      0.55      5001
          1       0.60      0.64      0.62      5423

avg / total       0.59      0.59      0.59     10424

Predict 1 percent: 0.5609171143514965
Label 1 percent: 0.5203389830508475
AUC: 0.622
Accuracy: 0.5876822716807367


In [2]:
lgb_feats = con_feats + cat_feats + bin_feats
categorical_feature_list = cat_feats + bin_feats

import lightgbm as lgb
lgb_params ={
    'objective':'binary',
    'boosting_type': 'gbdt',
    'metric':'auc',
    'learning_rate': 0.01,
    'num_leaves': 5,
    'max_depth': 4,
    'min_data_in_leaf': 100,
    'bagging_fraction': 0.8,
    'feature_fraction':0.8,
    'bagging_freq':10,
    'lambda_l1':0.2,
    'lambda_l2':0.2,
    'scale_pos_weight':1,
}

NameError: name 'con_feats' is not defined

In [ ]:
!pip install lightgbm